In [ ]:
# ─── imports ───────────────────────────────────────────────────────────────
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import deque

plt.style.use("default")
plt.rcParams["axes.facecolor"]   = "white"
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["grid.color"]       = "white"

# ─── incremental helpers ──────────────────────────────────────────────────
class IncWMA:
    def __init__(self, n: int):
        self.n = max(1, n)
        self.buf = deque(maxlen=self.n)
        self.w   = np.arange(1, self.n + 1, dtype=float)
        self.S   = self.w.sum()

    def update(self, x: float) -> float:
        self.buf.append(x)
        if len(self.buf) < self.n:          # warm-up: just echo price
            return x
        arr = np.fromiter(self.buf, float, len(self.buf))
        return float((self.w * arr).sum() / self.S)

class IncHMA:
    """Streaming Hull-MA identical to batch HMA."""
    def __init__(self, n: int):
        self.w_full  = IncWMA(n)
        self.w_half  = IncWMA(max(1, n // 2))
        self.w_final = IncWMA(max(1, int(np.sqrt(n))))

    def update(self, price: float) -> float:
        full  = self.w_full.update(price)
        half  = self.w_half.update(price)
        synth = 2.0 * half - full
        return self.w_final.update(synth)

class IncKalman:
    def __init__(self, R=0.075, Ql=4e-3, Qt=1e-5):
        self.F = np.array([[1, 1],
                           [0, 1]], float)
        self.H = np.array([[1, 0]], float)
        self.Q = np.diag([Ql, Qt])
        self.R = R
        self.s = None                    # [level, velocity]
        self.P = np.eye(2)

    def update(self, x: float) -> float:
        if self.s is None:               # first tick
            self.s = np.array([x, 0.0], float)
            return x
        # predict
        self.s = self.F @ self.s
        self.P = self.F @ self.P @ self.F.T + self.Q
        # update
        y   = x - (self.H @ self.s)[0]
        S   = (self.H @ self.P @ self.H.T)[0, 0] + self.R
        K   = (self.P @ self.H.T) / S
        self.s += K.flatten() * y
        self.P  = (np.eye(2) - K @ self.H) @ self.P
        return self.s[0]

# ─── BaseModelTrader (ultra-fast incremental, sign-reversed) ──────────────
class BaseModelTrader:
    """
    Streaming version of the batch HMA→Kalman→buffer→gradient model.
    Internal signal: +1 = **short**, -1 = **long** (reversed mapping).
    """
    def __init__(self,
                 hma_period=100,
                 N_buffer=8,
                 X_confirm=2,
                 R=0.075, Ql=4e-3, Qt=1e-5,
                 grad_pos=1e-3, grad_neg=-1e-3,
                 capital_per_inst=10_000):
        # hyper-params
        self.hma_period  = hma_period
        self.N_buffer    = N_buffer
        self.X_confirm   = X_confirm
        self.R, self.Ql, self.Qt = R, Ql, Qt
        self.grad_pos, self.grad_neg = grad_pos, grad_neg
        self.capital_per_inst = capital_per_inst
        # lazy init flag
        self.ready = False

    def _init(self, nInst, first_prices):
        self.hma   = [IncHMA(self.hma_period)           for _ in range(nInst)]
        self.kal   = [IncKalman(self.R, self.Ql, self.Qt) for _ in range(nInst)]
        self.buf_state = [(0, 0, 0) for _ in range(nInst)]  # (state, same, opp)
        self.kal_win   = [deque(maxlen=self.N_buffer) for _ in range(nInst)]
        self.prev_sig  = [0] * nInst
        self.prev_pos  = np.zeros(nInst, int)

        # prime with first prices so warm-up matches batch
        for i, p in enumerate(first_prices):
            h = self.hma[i].update(p)
            k = self.kal[i].update(p)
            raw = 1 if h > k else -1
            self.buf_state[i] = (raw, 1, 0)
            self.prev_sig[i]  = raw
            self.kal_win[i].append(k)
            max_sh = int(self.capital_per_inst // p) if p > 0 else 0
            self.prev_pos[i] = -raw * max_sh          # sign-reversed
        self.ready = True

    # ------------------------------------------------------------------
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        """
        prcSoFar shape = (nInst, t_seen) ; returns positions per instrument.
        """
        nInst = prcSoFar.shape[0]
        prices_now = prcSoFar[:, -1]

        if not self.ready:
            self._init(nInst, prices_now)

        out_pos = self.prev_pos.copy()

        for i in range(nInst):
            price  = prices_now[i]
            h_val  = self.hma[i].update(price)
            k_val  = self.kal[i].update(price)
            raw    = 1 if h_val > k_val else -1

            # buffer filter
            state, same, opp = self.buf_state[i]
            if raw == state:
                same += 1; opp = 0
            else:
                opp += 1
                if same >= self.N_buffer or opp >= self.X_confirm:
                    state, same, opp = raw, 1, 0
                else:
                    same = 0
            self.buf_state[i] = (state, same, opp)
            buf_sig = state

            # gradient filter
            w = self.kal_win[i]
            w.append(k_val)
            sig_prev = self.prev_sig[i]
            if buf_sig != sig_prev and len(w) >= 2:
                y     = np.fromiter(w, float)
                slope = np.polyfit(np.arange(len(y)), y, 1)[0] / (y.mean() or 1)
                sig   = sig_prev if (self.grad_neg <= slope <= self.grad_pos) else buf_sig
            else:
                sig = sig_prev

            # trade only on switch
            if sig != sig_prev:
                max_sh = int(self.capital_per_inst // price) if price > 0 else 0
                out_pos[i]       = -sig * max_sh      # sign-reversed
                self.prev_sig[i] = sig
                self.prev_pos[i] = out_pos[i]

        return out_pos


# ─── simulation & plotting ────────────────────────────────────────────────
# 1) load prices (file = whitespace rows oldest→newest)
prices = pd.read_csv("prices.txt", sep=r"\s+", header=None).values.T  # shape (nInst×T)
nInst, T = prices.shape

# choose window to display
t1, t2 = 250, 700   # inclusive

# 2) run the model
trader = BaseModelTrader()            # default hyper-params
positions = np.zeros((nInst, T), int)
for t in range(T):
    positions[:, t] = trader.Alg(prices[:, : t + 1])

# 3) plot each instrument
time = np.arange(t1, t2 + 1)
for inst in range(nInst):
    price_seg = prices  [inst, t1:t2+1]
    pos_seg   = positions[inst, t1:t2+1]

    longs  = pos_seg > 0
    shorts = pos_seg < 0

    fig, ax = plt.subplots(figsize=(12, 4), facecolor="white")
    ax.set_facecolor("white")
    ax.grid(False)

    # price line
    ax.plot(time, price_seg, color="black", linewidth=1)

    # thick black vertical separator every bar
    ax.vlines(time, ymin=price_seg.min(), ymax=price_seg.max(),
              color="black", linewidth=1, alpha=0.3)

    # scatter long/short points
    ax.scatter(time[longs],  price_seg[longs],  color="green", marker="o", s=30, label="Long")
    ax.scatter(time[shorts], price_seg[shorts], color="red",   marker="o", s=30, label="Short")

    ax.set_title(f"Instrument {inst}  (t={t1}–{t2})")
    ax.set_xlabel("Time step")
    ax.set_ylabel("Price")
    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.show()


In [ ]:
#!/usr/bin/env python3
"""
base_model_trader_with_metrics.py
─────────────────────────────────────────
Standalone script that:

1. Uses the incremental BaseModelTrader (HMA→Kalman→buffer→gradient).
2. Loads `prices.txt`, runs the trader on all instruments over a time window.
3. Plots each instrument individually with:
   • Price series + scatter for long (green) / short (red)
   • Cumulative PnL plot (big, below price)
   • Confidence level plot (big, below PnL)
4. Applies a commission cost of 0.0005 per dollar traded to the PnL calculation.
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import deque
from pathlib import Path
from typing import Tuple

plt.style.use("default")
plt.rcParams["axes.facecolor"] = "white"
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["grid.color"] = "white"

# ─── incremental helpers ──────────────────────────────────────────────────
class IncWMA:
    def __init__(self, n: int):
        self.n = max(1, n)
        self.buf = deque(maxlen=self.n)
        self.w = np.arange(1, self.n + 1, dtype=float)
        self.S = self.w.sum()
    def update(self, x: float) -> float:
        self.buf.append(x)
        if len(self.buf) < self.n:
            return x
        arr = np.fromiter(self.buf, float, len(self.buf))
        return float((self.w * arr).sum() / self.S)

class IncHMA:
    def __init__(self, n: int):
        self.w_full = IncWMA(n)
        self.w_half = IncWMA(max(1, n // 2))
        self.w_final = IncWMA(max(1, int(np.sqrt(n))))
    def update(self, price: float) -> float:
        full = self.w_full.update(price)
        half = self.w_half.update(price)
        synth = 2.0 * half - full
        return self.w_final.update(synth)

class IncKalman:
    def __init__(self, R=0.075, Ql=4e-3, Qt=1e-5):
        self.F = np.array([[1,1],[0,1]], float)
        self.H = np.array([[1,0]], float)
        self.Q = np.diag([Ql, Qt])
        self.R = R
        self.s = None
        self.P = np.eye(2)
    def update(self, x: float) -> float:
        if self.s is None:
            self.s = np.array([x, 0.0], float)
            return x
        self.s = self.F @ self.s
        self.P = self.F @ self.P @ self.F.T + self.Q
        y = x - (self.H @ self.s)[0]
        S = (self.H @ self.P @ self.H.T)[0,0] + self.R
        K = (self.P @ self.H.T) / S
        self.s += K.flatten() * y
        self.P = (np.eye(2) - K @ self.H) @ self.P
        return self.s[0]

# ─── BaseModelTrader with confidence output ─────────────────────────────────
class BaseModelTrader:
    def __init__(self,
                 hma_period=100,
                 N_buffer=8,
                 X_confirm=2,
                 R=0.075, Ql=4e-3, Qt=1e-5,
                 grad_pos=1e-3, grad_neg=-1e-3,
                 capital_per_inst=10_000,
                 commission=0.0005):
        self.hma_period = hma_period
        self.N_buffer = N_buffer
        self.X_confirm = X_confirm
        self.R, self.Ql, self.Qt = R, Ql, Qt
        self.grad_pos, self.grad_neg = grad_pos, grad_neg
        self.capital_per_inst = capital_per_inst
        self.commission = commission
        self.ready = False
    def _init(self, nInst: int, first_prices: np.ndarray):
        self.hma = [IncHMA(self.hma_period) for _ in range(nInst)]
        self.kal = [IncKalman(self.R, self.Ql, self.Qt) for _ in range(nInst)]
        self.buf = [(0,0,0) for _ in range(nInst)]
        self.win = [deque(maxlen=self.N_buffer) for _ in range(nInst)]
        self.sig = [0]*nInst
        self.pos = np.zeros(nInst, int)
        for i, p in enumerate(first_prices):
            h = self.hma[i].update(p)
            k = self.kal[i].update(p)
            raw = 1 if h > k else -1
            self.buf[i] = (raw, 1, 0)
            self.sig[i] = raw
            self.win[i].append(k)
            max_sh = int(self.capital_per_inst // p)
            self.pos[i] = -raw * max_sh
        self.ready = True
    def Alg(self, prcSoFar: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        nInst, _ = prcSoFar.shape
        prices_now = prcSoFar[:, -1]
        if not self.ready:
            self._init(nInst, prices_now)
        out_pos = self.pos.copy()
        conf = np.zeros(nInst, float)
        for i in range(nInst):
            price = prices_now[i]
            h = self.hma[i].update(price)
            k = self.kal[i].update(price)
            raw = 1 if h > k else -1
            conf[i] = min(abs(h - k) / price, 1.0)
            st, same, opp = self.buf[i]
            if raw == st:
                same += 1; opp = 0
            else:
                opp += 1
                if same >= self.N_buffer or opp >= self.X_confirm:
                    st, same, opp = raw, 1, 0
                else:
                    same = 0
            self.buf[i] = (st, same, opp)
            buf_sig = st
            w = self.win[i]; w.append(k)
            prev = self.sig[i]
            if buf_sig != prev and len(w) >= 2:
                y = np.fromiter(w, float)
                slope = np.polyfit(np.arange(len(y)), y, 1)[0] / (y.mean() or 1)
                sig = prev if (self.grad_neg <= slope <= self.grad_pos) else buf_sig
            else:
                sig = prev
            if sig != prev:
                max_sh = int(self.capital_per_inst // price)
                out_pos[i] = -sig * max_sh
                self.sig[i] = sig
                self.pos[i] = out_pos[i]
        return out_pos, conf

# ─── load data & simulate ─────────────────────────────────────────
prices = pd.read_csv(Path("prices.txt"), sep=r"\s+", header=None).values.T
nInst, T = prices.shape
# time window
t1, t2 = 250, 700
trader = BaseModelTrader()
pos_arr = np.zeros((nInst, T), int)
conf_arr = np.zeros((nInst, T), float)
for t in range(T):
    pos_arr[:, t], conf_arr[:, t] = trader.Alg(prices[:, :t+1])
# compute pnl with commission
trade_pnl = np.zeros((T, nInst))
comm_pnl  = np.zeros((T, nInst))
for t in range(1, T):
    prev_pos = pos_arr[:, t-1]
    curr_pos = pos_arr[:, t]
    price_diff = prices[:, t] - prices[:, t-1]
    trade_pnl[t] = prev_pos * price_diff
    comm_pnl[t]  = -trader.commission * np.abs(curr_pos - prev_pos) * prices[:, t]
cum_pnl = np.cumsum(trade_pnl + comm_pnl, axis=0)

# ─── plotting per instrument ───────────────────────────────────────
for i in range(nInst):
    time = np.arange(t1, t2+1)
    price_seg = prices[i, t1:t2+1]
    pos_seg   = pos_arr[i, t1:t2+1]
    pnl_seg   = cum_pnl[t1:t2+1, i]
    conf_seg  = conf_arr[i, t1:t2+1]
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12,12), sharex=True)
    ax1.plot(time, price_seg, color='black')
    ax1.scatter(time[pos_seg>0], price_seg[pos_seg>0], color='green', label='Long')
    ax1.scatter(time[pos_seg<0], price_seg[pos_seg<0], color='red', label='Short')
    ax1.set_ylabel('Price'); ax1.legend()
    ax2.plot(time, pnl_seg)
    ax2.set_ylabel('Cumulative PnL'); ax2.axhline(0, color='black', linewidth=0.5)
    ax3.plot(time, conf_seg)
    ax3.set_ylabel('Confidence'); ax3.set_ylim(0,1); ax3.set_xlabel('Time')
    fig.suptitle(f"Instrument {i:02d}  (t={t1}-{t2})")
    plt.tight_layout(rect=[0,0.03,1,0.95])
    plt.show()
